# Preprocesamiento de Datos

### Dataset: Credit Card Fraud Detection
**Link:https://www.kaggle.com/c/ieee-fraud-detection/overview**



**Consideraciones:**

-No solo contiene transferencia de dinero sino tambien compra de regalos(como reservas de hoteles) por eso la doble direccion de correo y algunas no tiene R_emaildomain porque no son para nadie.

-Los codigos de productos no son necesariamente fisicos tambien pueden ser servicios.

-Las direcciones son del comprador el addr1=region de facturacion , addr2=pais de facturacion

-Las variables V tienen informacion como: Por ejemplo, cuántas veces apareció la tarjeta de pago asociada con una IP y un correo electrónico o dirección en un rango de tiempo de 24 horas,Todas las características de Vesta se derivaron como numéricas. Algunas de ellas son recuentos de órdenes dentro de una agrupación, un período de tiempo o condición, por lo que el valor es finito y tiene orden (o clasificación). No recomendaría tratar ninguna de ellas. como categórico. Si alguno de ellos resultó en binario por casualidad, tal vez valga la pena intentarlo

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk

## Lectura de archivos

In [2]:
#Lectura de los datos del archivo csv
data = pd.read_csv('Data/credit_dataset_2_transaccion.csv')


#Lectura de los datos del archivo csv
ident = pd.read_csv('Data/credit_dataset_2_identity.csv')

In [3]:
data.head(5)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Verificacion de columnas categoricas

In [3]:
# Seleccionar columnas numéricas
numeric_columns = data.select_dtypes(include=[np.number]).columns
print("Columnas numéricas:")
print(numeric_columns)
print(f"Total de columnas numéricas: {len(numeric_columns)}")

# Seleccionar columnas categóricas
categorical_columns = data.select_dtypes(include=['object']).columns
print("\nColumnas categóricas:")
print(categorical_columns)
print(f"Total de columnas categóricas: {len(categorical_columns)}")

Columnas numéricas:
Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'card1',
       'card2', 'card3', 'card5', 'addr1', 'addr2',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=380)
Total de columnas numéricas: 380

Columnas categóricas:
Index(['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1',
       'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9'],
      dtype='object')
Total de columnas categóricas: 14


## Aplicacion de ONE-HOT ENCODING a las variables categoricas

In [4]:
# Variables a aplicar on/e-hot encoding CARD4 CARD6 PRODUCTCD
one_hot_columns = ['card4', 'card6','M4', 'ProductCD']

# Aplicar one-hot encoding a las columnas especificadas

for col in one_hot_columns:
    data[col]=data[col].fillna('missing')

    # Crear columnas dummy para cada categoría en la columna
    dummies = pd.get_dummies(data[col], prefix=col)
    
    # Concatenar las columnas dummy al DataFrame original
    data = pd.concat([data, dummies], axis=1)
    
    # Eliminar la columna original
    data.drop(col, axis=1, inplace=True)

## Agrupacion de campos y luego codificacion con ONE-HOT ENCODING

In [6]:
#Se agrupa los proveedores de correo electrónico en categorías mas especificas para aplicar target encoding
# Se define un diccionario que mapea los dominios de correo electrónico a sus respectivos proveedores

email_provider_map = {
    # Google
    'gmail.com': 'google', 'gmail': 'google',
    
    # Microsoft
    'hotmail.com': 'microsoft', 'hotmail.es': 'microsoft', 'hotmail.fr': 'microsoft', 'hotmail.de': 'microsoft',
    'hotmail.co.uk': 'microsoft', 'live.com': 'microsoft', 'live.com.mx': 'microsoft', 'live.fr': 'microsoft',
    'outlook.com': 'microsoft', 'outlook.es': 'microsoft', 'msn.com': 'microsoft',

    # Yahoo
    'yahoo.com': 'yahoo', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo', 'yahoo.de': 'yahoo',
    'yahoo.es': 'yahoo', 'yahoo.co.uk': 'yahoo', 'yahoo.co.jp': 'yahoo',
    'ymail.com': 'yahoo', 'rocketmail.com': 'yahoo',

    # Apple
    'icloud.com': 'apple', 'me.com': 'apple', 'mac.com': 'apple',

    # AOL
    'aol.com': 'aol', 'aim.com': 'aol',

    # ProtonMail
    'protonmail.com': 'secure',

    # GMX / Web.de (Alemania)
    'gmx.de': 'gmx', 'web.de': 'gmx',

    # ISP-based
    'comcast.net': 'isp', 'verizon.net': 'isp', 'att.net': 'isp',
    'bellsouth.net': 'isp', 'sbcglobal.net': 'isp', 'cox.net': 'isp',
    'charter.net': 'isp', 'optonline.net': 'isp', 'earthlink.net': 'isp',
    'embarqmail.com': 'isp', 'frontiernet.net': 'isp', 'frontier.com': 'isp',
    'windstream.net': 'isp', 'twc.com': 'isp', 'roadrunner.com': 'isp',
    'cableone.net': 'isp', 'centurylink.net': 'isp', 'ptd.net': 'isp',
    'netzero.net': 'isp', 'netzero.com': 'isp', 'sc.rr.com': 'isp',
    'cfl.rr.com': 'isp', 'q.com': 'isp',

    # Universidades / Instituciones
    'scranton.edu': 'edu',

    # Otros conocidos
    'mail.com': 'other', 'juno.com': 'other', 'servicios-ta.com': 'other', 'anonymous.com': 'anonymous',
    'prodigy.net.mx': 'other'
}

In [7]:
print(data.value_counts('P_emaildomain'))
print(data.value_counts('R_emaildomain'))


P_emaildomain
gmail.com           228355
yahoo.com           100934
hotmail.com          45250
anonymous.com        36998
aol.com              28289
comcast.net           7888
icloud.com            6267
outlook.com           5096
msn.com               4092
att.net               4033
live.com              3041
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
yahoo.com.mx          1543
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
live.com.mx            749
rocketmail.com         664
mail.com               559
earthlink.net          514
gmail                  496
outlook.es             438
mac.com                436
juno.com               322
aim.com                315
hotmail.es             305
roadrunner.com         305
windstream.net         305
hotmail.fr             295
frontier.com           280
embarqmail.com         260
web.de                 240
netzero.com   

In [8]:
# Variables a agrupar por categorias mas grandes para luego aplicar one hot encodin
target_encoding_columns = [ 'P_emaildomain', 'R_emaildomain']

# Aplicar target encoding a las columnas especificadas

data['P_emaildomain'] = data['P_emaildomain'].map(email_provider_map).fillna('missing')
data['R_emaildomain'] = data['R_emaildomain'].map(email_provider_map).fillna('missing')


for col in target_encoding_columns:

    # Crear columnas dummy para cada categoría en la columna
    dummies = pd.get_dummies(data[col], prefix=col)
    
    # Concatenar las columnas dummy al DataFrame original
    data = pd.concat([data, dummies], axis=1)
    
    # Eliminar la columna original
    data.drop(col, axis=1, inplace=True)



In [9]:
print(data.columns.tolist())

['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V9

## Codificacion de categorias binarias

In [10]:
# Variables a aplicar binarization
binarization_columns = ['M1', 'M2', 'M3',  'M5', 'M6', 'M7', 'M8', 'M9']

# Aplicar binarization a las columnas especificadas

for col in binarization_columns:
    data[col] = data[col].map({'T': 1, 'F': 0})
    data[col] = data[col].fillna(data[col].mode()[0])

### Eliminacion de valores con mas del 80% de valores nulos


In [11]:
threshold = 0.8  # Umbral de 90% de valores faltantes
data = data.dropna(thresh=int(threshold * len(data)), axis=1)

In [12]:
data.columns.tolist()

['TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'card1',
 'card2',
 'card3',
 'card5',
 'addr1',
 'addr2',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D10',
 'D15',
 'M1',
 'M2',
 'M3',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V65',
 'V66',
 'V67',
 'V68',
 'V69',
 'V70',
 'V71',
 'V72',
 'V73',
 'V74',
 'V75',
 'V76',
 'V77',
 'V78',
 'V79',
 'V80',
 'V81',
 'V82',
 'V83',
 'V84',
 'V85',
 'V86',
 'V87',
 'V88',
 'V89',
 'V90',
 'V91',
 'V92',
 'V93',
 'V94',
 'V95',
 'V96',
 'V97',
 'V98',
 'V99',
 'V100',
 'V101',
 'V102',
 'V103',
 'V104',
 'V105',
 'V106',
 'V107',
 'V108',
 'V109',
 'V110',
 'V111',
 'V112',
 'V113',
 'V11